## Reporting

In [1]:
from typing import Dict, List, Optional
from pydantic import BaseModel
from datetime import datetime

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
def system_prompt() -> str:
    """현재 타임스탬프를 포함한 시스템 프롬프트를 생성합니다."""
    now = datetime.now().isoformat()
    return f"""당신은 전문 연구원입니다. 오늘 날짜는 {now}입니다. 응답 시 다음 지침을 따르세요:
    - 지식 컷오프 이후의 주제에 대한 조사를 요청받을 수 있습니다. 사용자가 뉴스 내용을 제시했다면, 그것을 사실로 가정하세요.
    - 사용자는 매우 숙련된 분석가이므로 내용을 단순화할 필요 없이 가능한 한 자세하고 정확하게 응답하세요.
    - 체계적으로 정보를 정리하세요.
    - 사용자가 생각하지 못한 해결책을 제안하세요.
    - 적극적으로 사용자의 필요를 예측하고 대응하세요.
    - 사용자를 모든 분야의 전문가로 대우하세요.
    - 실수는 신뢰를 저하시킵니다. 정확하고 철저하게 응답하세요.
    - 상세한 설명을 제공하세요. 사용자는 많은 정보를 받아들일 수 있습니다.
    - 권위보다 논리적 근거를 우선하세요. 출처 자체는 중요하지 않습니다.
    - 기존의 통념뿐만 아니라 최신 기술과 반대 의견도 고려하세요.
    - 높은 수준의 추측이나 예측을 포함할 수 있습니다. 단, 이를 명확히 표시하세요."""

In [17]:
def write_final_report(
    prompt,
    learnings: List[str],
    visited_papers: List[str],
    model_name: str,
) -> str:
    """
    연구 결과를 바탕으로 최종 보고서를 생성

    """
    research_componant = ("\n".join(
        [f"<learning>\n{learning}\n</learning>" for learning in learnings]
    )).strip()

    report_prompt = PromptTemplate(
        input_variables=["prompt"],
        template="""
        ### System:
        {system_prompt}

        ### Instruction:
        Based on the research results, write a final report for the following user-provided prompt.
        The report should be detailed in Markdown format and exceed 6,000 characters.
        Include all learnings obtained from the research:\n\n
        <prompt>{prompt}</prompt>\n\n

        The following are all the learnings obtained from the research:\n\n<learnings>\n{research_componant}</learnings> 
        """
    ).partial(
        system_prompt=system_prompt(),
        research_componant=research_componant,
    )
    report_llm = ChatGoogleGenerativeAI(model=model_name)
    report_chain = report_prompt | report_llm | StrOutputParser()

    try:
        report = report_chain.invoke({"prompt": prompt})
        reference_section = "\n\n##Reference\n\n" + \
            "\n".join(f"- {ref}" for ref in visited_papers)
        return report + reference_section
    except Exception as e:
        print(f"Error generating report: {e}")
        return "Error generating report"

In [18]:
query = "RAG의 최근 동향을 알려줘."
feedback_questions = ['RAG의 어떤 측면에 대해서 알고 싶으신가요? (예: 기술적 동향, 시장 점유율, 투자 동향 등)',
                      '특정 기간(예: 지난 6개월, 최근 1년)에 대한 정보가 필요하신가요?',
                      'RAG에 대한 정보를 어떻게 활용하실 예정인가요?']
answers = ["기술적 동햑", "최근 1년간", "RAG 어떻게 활용되고 있는지를 조사해서 실무에 적용해볼 생각"]

combined_query = f"초기 질문: {query}\n"
for i in range(len(feedback_questions)):
    combined_query += f"\n{i+1}. 질문: {feedback_questions[i]}\n"
    combined_query += f"답변: {answers[i]}\n"
    
combined_query

'초기 질문: RAG의 최근 동향을 알려줘.\n\n1. 질문: RAG의 어떤 측면에 대해서 알고 싶으신가요? (예: 기술적 동향, 시장 점유율, 투자 동향 등)\n답변: 기술적 동햑\n\n2. 질문: 특정 기간(예: 지난 6개월, 최근 1년)에 대한 정보가 필요하신가요?\n답변: 최근 1년간\n\n3. 질문: RAG에 대한 정보를 어떻게 활용하실 예정인가요?\n답변: RAG 어떻게 활용되고 있는지를 조사해서 실무에 적용해볼 생각\n'

In [ ]:
report = write_final_report(
    prompt=combined_query,
    learnings=
)